# Prepocess

In [ ]:
# import os
# import numpy as np
# import pandas as pd
# import multiprocessing  # For parallel processing

# def process_npy_file(file_path):
#     """Processes a single .npy file and returns row and column counts."""
#     try:
#         data = np.load(file_path, mmap_mode='r')  # Use memory mapping if possible
#         rows = data.shape[0]
#         cols = data.shape[1] if data.ndim > 1 else 1
#         return rows, cols, os.path.basename(file_path) # Return filename as well
#     except (OSError, ValueError) as e:
#         print(f"Error processing {file_path}: {e}")
#         return None, None, os.path.basename(file_path)  # Return None for failed files
#     except Exception as e:
#         print(f"An unexpected error occurred processing {file_path}: {e}")
#         return None, None, os.path.basename(file_path)  # Return None for failed files


# def find_Row_Col(path, num_processes=None):
#     if not os.path.exists(path):
#         print(f"Error: Path '{path}' not found.")
#         return None

#     npy_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".npy")]

#     if num_processes is None:
#         num_processes = multiprocessing.cpu_count()  # Use all available cores

#     pool = multiprocessing.Pool(processes=num_processes)  # Create a process pool
#     results = pool.map(process_npy_file, npy_files)  # Process files in parallel
#     pool.close()  # Close the pool
#     pool.join()   # Wait for all processes to finish

#     lst_row = []
#     lst_column = []
#     filenames = []

#     for rows, cols, filename in results:
#         if rows is not None:  # Check if processing was successful
#             lst_row.append(rows)
#             lst_column.append(cols)
#             filenames.append(os.path.join(path,filename))

#     df = pd.DataFrame({'Rows': lst_row, 'Columns': lst_column}, index=filenames)
#     return df


# # Example usage (adjust num_processes if needed):
# train_path = "/kaggle/input/signal-fast-radio-burst-detection/train/train"
# test_path = "/kaggle/input/signal-fast-radio-burst-detection/test/test"

# train_df = find_Row_Col(train_path)
# test_df = find_Row_Col(test_path)

# if train_df is not None:
#     print("Train DataFrame:")
#     print(train_df)
#     train_df.to_csv("train_data.csv")
# else:
#     print("Error: Train DataFrame could not be created.")

# if test_df is not None:
#     print("\nTest DataFrame:")
#     print(test_df)
#     test_df.to_csv("test_data.csv")
# else:
#     print("Error: Test DataFrame could not be created.")

In [ ]:
# print("Rows")
# print(train_df.Rows.unique())
# print(test_df.Rows.unique())

In [ ]:
# print("Columns")
# print(train_df.Columns.unique())
# print(test_df.Columns.unique())

In [ ]:
# print("File_paths")
# print(train_df.index.unique())
# print(test_df.index.unique())

In [ ]:
# import numpy as np
# import os
# from PIL import Image
# from tqdm import tqdm
# import multiprocessing
# from functools import partial

# def process_single_array(array_data, file_base, array_index, output_dir):
#     """Processes a single array chunk."""
#     try:
#         current_array = array_data

#         if current_array.dtype != np.uint8:
#             min_val = np.min(current_array)
#             max_val = np.max(current_array)
#             if max_val - min_val != 0:
#                 current_array = ((current_array - min_val) / (max_val - min_val) * 255).astype(np.uint8)
#             else:
#                 current_array = np.zeros_like(current_array, dtype=np.uint8)

#         img = Image.fromarray(current_array)
#         if img.mode != 'L':
#             img = img.convert("L")
#         output_filename = f"{file_base}_{array_index}.jpg"
#         output_path = os.path.join(output_dir, output_filename)
#         img.save(output_path)
#         return True  # Indicate success

#     except Exception as e:
#         print(f"Error converting/saving {file_base}, part {array_index}: {e}")
#         import matplotlib.pyplot as plt
#         plt.imshow(current_array)
#         plt.title("Array before conversion")
#         plt.show()
#         raise  # Re-raise for full traceback
#         return False # Indicate failure


# def process_file(file_path, output_dir):  # output_dir is now passed directly
#     try:
#         array_list = np.load(file_path)
#         file_base = os.path.splitext(os.path.basename(file_path))[0]
#         success_count = 0

#         with tqdm(range(len(array_list) // 256), desc=f"Processing {file_base}", leave=False) as pbar:
#             for array_index in pbar:
#                 start_index = array_index * 256
#                 end_index = min((array_index + 1) * 256, len(array_list))
#                 current_array = array_list[start_index:end_index]

#                 if process_single_array(current_array, file_base, array_index, output_dir):
#                     success_count += 1
#         return success_count

#     except FileNotFoundError:
#         print(f"File not found: {file_path}")
#         return 0
#     except Exception as e:
#         print(f"Error processing file {file_path}: {e}")
#         return 0


# def process_data_multiprocessing(df, output_dir="output_images", num_processes=None):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     if num_processes is None:
#         num_processes = multiprocessing.cpu_count()

#     pool = multiprocessing.Pool(processes=num_processes)
#     total_files = len(df.index)
#     total_success = 0
#     total_chunks = 0

#     file_paths = list(df.index)  # Convert index to a list for consistent ordering

#     with tqdm(total=total_files, desc=f"Total Progress") as pbar_total:
#         for i, result in enumerate(pool.imap_unordered(partial(process_file, output_dir=output_dir), file_paths)):
#             total_success += result
#             pbar_total.update(1)

#             # Calculate total chunks using the *original* file path order:
#             file_path = file_paths[i]  # Use the correct index from the list
#             try:
#                 array_list_temp = np.load(file_path)
#                 total_chunks += len(array_list_temp) // 256
#             except Exception as e:
#                 print(f"Error loading {file_path}: {e}")
#                 pass

#     pool.close()
#     pool.join()

#     print(f"Successfully converted {total_success} chunks out of {total_chunks} total")


# process_data_multiprocessing(train_df, 'train', num_processes=8) # Example: Use 8 processes
# process_data_multiprocessing(test_df, 'test', num_processes=8) # Example: Use 8 processes

In [8]:
import os
import zipfile
import shutil

def zip_download_folder(download_folder_path, output_zip_path):
    try:
        # Check if the download folder exists
        if not os.path.exists(download_folder_path):
            raise FileNotFoundError(f"Download folder not found: {download_folder_path}")

        # Create a ZipFile object in write mode
        with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:  # ZIP_DEFLATED for compression
            for root, _, files in os.walk(download_folder_path): # Walk through all subdirectories
                for file in files:
                    file_path = os.path.join(root, file)
                    arc_name = os.path.relpath(file_path, download_folder_path)  # Get relative path for archive
                    zipf.write(file_path, arc_name)  # Write the file to the zip archive

        print(f"Download folder zipped successfully to: {output_zip_path}")

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An error occurred during zipping: {e}")


# # Example usage:
# download_folder = "/kaggle/working/test"  # Replace with the actual path to your download folder
# output_zip = "/kaggle/working/test.zip"  # Replace with the desired output zip file path

# zip_download_folder(download_folder, output_zip)

# download_folder = "/kaggle/working/train"  # Replace with the actual path to your download folder
# output_zip = "/kaggle/working/train.zip"  # Replace with the desired output zip file path

# zip_download_folder(download_folder, output_zip)

download_folder = "/kaggle/working/sorted_data"  # Replace with the actual path to your download folder
output_zip = "/kaggle/working/sorted_data.zip"  # Replace with the desired output zip file path

zip_download_folder(download_folder, output_zip)

Download folder zipped successfully to: /kaggle/working/sorted_data.zip


In [ ]:
# from IPython.display import FileLink
# FileLink('/kaggle/working/sorted_data.zip')

# train

In [1]:
import os
import pandas as pd

data_dir = "/kaggle/input/signal-fast-radio-burst-detection/train-labels-corrected/train"

dfs = []

# Loop through files in the directory
for idx, file in enumerate(os.listdir(data_dir)):  # Use enumerate to get an index
    if file.endswith(".csv"):  # Check if file is a CSV
        # print(file)
        # break
        file_path = os.path.join(data_dir, file)
        df = pd.read_csv(file_path)
        # print(file)
        # break
        file_name = file.strip("labels.csv")
        # Create a name column with "file_index_rowindex" format
        df["name"] = [f"{file_name}{i}.jpg" for i in range(len(df))]

        dfs.append(df)

# Merge all dataframes
merged_df = pd.concat(dfs, ignore_index=True)
merged_df.drop(columns =["Unnamed: 0","index"],inplace =True)
# Save the merged dataframe to a new CSV file
merged_df.to_csv("merged.csv", index=False)

print("All CSV files have been merged successfully into merged.csv")


All CSV files have been merged successfully into merged.csv


In [2]:
df = pd.read_csv("merged.csv")
df.fillna("NAN",inplace =True)
df

,labels,name
0,NAN,B1933+16_59475_68410_reduced_fc_0001023_0.jpg
1,NAN,B1933+16_59475_68410_reduced_fc_0001023_1.jpg
2,NAN,B1933+16_59475_68410_reduced_fc_0001023_2.jpg
3,NAN,B1933+16_59475_68410_reduced_fc_0001023_3.jpg
4,NAN,B1933+16_59475_68410_reduced_fc_0001023_4.jpg
...,...,...
96420,Narrow,B0531+21_58713_43190_reduced_fc_0007167_1019.jpg
96421,Narrow,B0531+21_58713_43190_reduced_fc_0007167_1020.jpg
96422,Narrow,B0531+21_58713_43190_reduced_fc_0007167_1021.jpg
96423,Narrow,B0531+21_58713_43190_reduced_fc_0007167_1022.jpg


In [3]:
# df = df[~df.labels.isin(['Unlabeled','Uncertain'])]
for l in df.labels.unique():
    print(len(df[df.labels == l]),l)

62988 NAN
3847 Pulse
3201 Broad
9 Broad+Pulse
11842 Unlabeled
5721 Uncertain
8582 Narrow
60 Narrow+Pulse
138 Narrow+Broad
37 Unknown+Pulse


In [4]:
# from imblearn.over_sampling import SMOTE
X = df.name
y = df.labels
# X_resampled, y_resampled = SMOTE().fit_resample(X, y)


In [5]:
from sklearn.model_selection import train_test_split

# Assuming you have X (features) and y (labels) variables already defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)


Training data shape: (77140,)
Test data shape: (19285,)


In [6]:
import os
import shutil
import pandas as pd

def makeDataset(X,y , folder_name = "train"):

    df = pd.concat([X, y], axis=1)
    # Define paths
    data_dir = "/kaggle/input/preprocess/train (1)"  # Source directory
    output_dir = f"/kaggle/working/sorted_data/{folder_name}"  # Destination directory
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    
    # Iterate over unique labels and copy files
    for label in y.unique():
        label_dir = os.path.join(output_dir, str(label))  # Create folder for label
        os.makedirs(label_dir, exist_ok=True)
    
        # Select rows with the current label
        x_df = df[df["labels"] == label]
    
        # Copy files into respective label folders
        for _, row in x_df.iterrows():
            file_name = row["name"]  # Assuming this is the filename
            src_path = os.path.join(data_dir, file_name)  # Source file path
            dst_path = os.path.join(label_dir, file_name)  # Destination path
            
            if os.path.exists(src_path):  # Ensure file exists before copying
                shutil.copy(src_path, dst_path)
    
    print("Files successfully copied into their respective label folders!")


In [7]:
makeDataset(X_train,y_train,'train')
makeDataset(X_test,y_test,'val')


Files successfully copied into their respective label folders!
Files successfully copied into their respective label folders!


In [ ]:
# import shutil
# import os

# # Define the folder path
# folder_path = "/kaggle/working/sorted_data"

# # Check if the folder exists before attempting to delete
# if os.path.exists(folder_path) and os.path.isdir(folder_path):
#     shutil.rmtree(folder_path)
#     print(f"Folder {folder_path} and all its contents have been deleted.")
# else:
#     print(f"The folder {folder_path} does not exist.")


In [10]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.6/914.6 kB 21.1 MB/s eta 0:00:0000:01


In [11]:
from ultralytics import YOLO

# Load a YOLOv8 model pretrained on ImageNet
model = YOLO("yolo11l-cls")  # Use "yolov8s-cls.pt" for a larger model



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 27.2M/27.2M [00:00<00:00, 253MB/s]


In [12]:
model.train(data="/kaggle/working/sorted_data", epochs=10, imgsz=224)
# # Run inference on an image
# results = model("example.jpg")  # Replace with your image path

# # Print the predicted class
# print(results)|


Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolo11l-cls.pt, data=/kaggle/working/sorted_data, epochs=10, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

100%|██████████| 5.35M/5.35M [00:00<00:00, 106MB/s]


AMP: checks passed ✅


train: Scanning /kaggle/working/sorted_data/train... 77140 images, 0 corrupt: 100%|██████████| 77140/77140 [00:35<00:00, 2159.53it/s]


train: New cache created: /kaggle/working/sorted_data/train.cache


...

val: New cache created: /kaggle/working/sorted_data/val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 82 weight(decay=0.0), 83 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 224 train, 224 val
Using 2 dataloader workers
Logging results to runs/classify/train
Starting training for 10 epochs...

      Epoch    GPU_mem       loss  Instances       Size


       1/10      1.06G      2.265         16        224:   0%|          | 4/4822 [00:01<15:52,  5.06it/s]

       1/10      1.13G      2.295         16        224:   0%|          | 9/4822 [00:01<09:35,  8.37it/s]
100%|██████████| 755k/755k [00:00<00:00, 25.9MB/s]
               classes   top1_acc   top5_acc: 100%|██████████| 603/603 [00:35<00:00, 17.17it/s]

                   all      0.751      0.999


AttributeError: module 'ray.train._internal.session' has no attribute '_get_session'